# NimAgent
This class represents our agent, the one that uses the weights that are updated through an ES-startegy

In [ ]:
class NimAgent:
    rules = list()
    weights = list()

    def __init__(self, weights, rules):
        self.weights = weights
        self.rules = rules
    
    def choose_move(self, state:Nim) -> Nimply:
        # More efficient working on ndarray
        np_weights = np.array(self.weights)
        indexes = np.argsort(np_weights)
        priority_queue = np.array(self.rules)[indexes][::-1]

        move = None
        for rule in priority_queue:
            ply = rule(state)
            if ply != None:
                move = ply
                break 
        
        return move